<a href="https://colab.research.google.com/github/luguzman/NLP/blob/main/BERT_squad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fase 1: Importar dependencias

In [1]:
!pip install sentencepiece
#!pip install tf-models-official
!pip install tf-models-nightly # mejor instalar la versión en desarrollo
!pip install tf-nightly

     |████████████████████████████████| 1.2MB 6.4MB/s 
     |████████████████████████████████| 1.5MB 6.6MB/s 
     |████████████████████████████████| 407.8MB 32kB/s 
     |████████████████████████████████| 71kB 7.6MB/s 
     |████████████████████████████████| 645kB 55.1MB/s 
     |████████████████████████████████| 4.4MB 58.1MB/s 
     |████████████████████████████████| 174kB 47.5MB/s 
     |████████████████████████████████| 51kB 5.9MB/s 
     |████████████████████████████████| 37.6MB 124kB/s 
     |████████████████████████████████| 102kB 8.2MB/s 
     |████████████████████████████████| 706kB 54.0MB/s 
     |████████████████████████████████| 358kB 58.6MB/s 
     |████████████████████████████████| 4.0MB 50.6MB/s 
     |████████████████████████████████| 471kB 54.8MB/s 
     |████████████████████████████████| 5.9MB 52.6MB/s 
     |████████████████████████████████| 4.0MB 58.3MB/s 
     |████████████████████████████████| 3.8MB 52.1MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp3

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

'2.4.0-dev20200822'

In [ ]:
import tensorflow_hub as hub        # librería que nos ayuda a importar modelos con pesos ya almacenados

from official.nlp.bert.tokenization import FullTokenizer        # Es doonde esta todo lo del módulo de desarrollo de tensorflow
from official.nlp.bert.input_pipeline import create_squad_dataset    # Nos va a peritir crear el dataset de squad a partir de importar esos ficheros que descargue y coloque en mi google drive
from official.nlp.data.squad_lib import generate_tf_record_from_json_file   # Creará un fichero intermedio a partir del dataset, que es el llamada tf_record

from official.nlp import optimization   # Mejora del optimizador Adam que encaja perfecto con BERT

from official.nlp.data.squad_lib import read_squad_examples     # Basicamente es para leer los ficheros de evaluación y que podamos ver que también ha sido nuestro modole para resolver los problemas de squad
from official.nlp.data.squad_lib import FeatureWriter           # Herramienta que nos ayudará a tarabajar la salida de BERT y validarlo como parte del proceso final 
from official.nlp.data.squad_lib import convert_examples_to_features    # Herramienta que nos ayudará a tarabajar la salida de BERT y validarlo como parte del proceso final
from official.nlp.data.squad_lib import write_predictions       # nos permitirá escribir las predicciones en un formato json

In [ ]:
import numpy as np
import math
import random
import time
import json
import collections
import os

from google.colab import drive

# Fase 2: Preprocesado de Datos

In [ ]:
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# Creamos un fichero  json intermedio que contendra el conjunto de entrenamiento junto con el vocabulario 
input_meta_data = generate_tf_record_from_json_file(
    "/content/drive/My Drive/Curso de NLP/BERT/squad_data/train-v1.1.json", # Acceso al fichero de entrenamiento
    "/content/drive/My Drive/Curso de NLP/BERT/squad_data/vocab.txt",       # La ruta al fichero de vocabulario 
    "/content/drive/My Drive/Curso de NLP/BERT/squad_data/train-v1.1.tf_record") # Indicamos el path donde queremos que se guarde este fichero tensorflow record

In [ ]:
# Almacenamos los metadatos para entrenar 
with tf.io.gfile.GFile("/content/drive/My Drive/Curso de NLP/BERT/squad_data/train_meta_data", "w") as writer:
    writer.write(json.dumps(input_meta_data, indent=4) + "\n") 
# \n: es simplemente para no tener problemas de lectura pues los ficheros deberían acabar siempre con un intro al final  

In [ ]:
# Creamos el dataset a partir de todos estos datos volcados, de estos tf_records
BATCH_SIZE = 4  # Notemos que aquí tenemos un pequeño problema y es que al crear este dataset yo no puedo indicar un tamaño de lote demasiado
# grande y es que aparentemente Google dice que utilizar un un batch_size demasiado grande sería bastante complicado. La razón es que las entradas
# son mucho más grandes, mucho más largas que las que hemos utilizado en formato de tweet. Recordemos que un tweet caben 140 caracteres. Aquí
# podriamos tener frases realmente largas para la tarea de preguntas y respuestas. Recordemos también que la primera entrada es todo un texto, 
# podría ser la página entra de un libro que contenga información de un tema.

train_dataset = create_squad_dataset(
    "/content/drive/My Drive/Curso de NLP/BERT/squad_data/train-v1.1.tf_record",
    input_meta_data['max_seq_length'], # sequencia más larga con la que nos encontramos que en este caso es de 384
    BATCH_SIZE,
    is_training=True)  # A la hr de crear el conjunto squad el de entrenamiento y el de testing es diferente

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: '<' not supported between instances of 'str' and 'Literal'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: '<' not supported between instances of 'str' and 'Literal'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: '<' not supported between instances of 'str' and 'Literal'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


# Fase 3: Construcción del modelo

## Capa Squad

Para cada uno de los vectores de salida que me devuelve BERT vamos a aplicarle una capa densa que se conectará con dos unidades(neuronas) en la capa de salida. Que van a ser para cada token el 1° número, qué tan problable es que la respuesta empiece en dicho token y el segundo número nos dirá lo mismo: la verosimilitud de que ese token sea el final de frase que responde a esa pregunta. Dicho esto habrá que re-dimensionarlo y colocarlo de modo que si tenemos un párrafo de 200 palabras en lugar de tener 200 pares de valores. Score de ser el inicio y score de ser el final, voy preferir tener 200 valores en dos listas separadas, ie, una lista en donde vendrá el score por palabra de ser el inicio de frase y de la misma manera la segunda pero considerando el score de que se el final de la frase.

In [ ]:
class BertSquadLayer(tf.keras.layers.Layer):

  def __init__(self):
    # Inicializamos la super clase. de este modo ya estarán creadas las variables y los metodos pertenecientes a la super clase.
    super(BertSquadLayer, self).__init__()
    # Creamos la capa densa de la que hemos platicado.
    self.final_dense = tf.keras.layers.Dense(
        units=2,
        kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02))  # Google recomiendo que a la hr de hacer find tuning con BERT utilicemos una normal truncada con un valor pequeño de la desviación estándar

    # inputs: son los valores que saldran de la capa de BERT
  def call(self, inputs):
    logits = self.final_dense(inputs) # (batch_size, seq_len, 2)

    # Quiero tener un par de listas dodne cada una de ellas tenga esos 200 valores pro separado, por lo que tenemos que re-dimensionar y cortar el tensor
    logits = tf.transpose(logits, [2, 0, 1]) # (2, batch_size, seq_len) pos 2: que tenía el score de ser inicio y final será lo primero, pos 0: lote de entrenamiento pos1:dimensión que se corresponde con las palabras (longitud de la secuencia)
    unstacked_logits = tf.unstack(logits, axis=0) # [(batch_size, seq_len), (batch_size, seq_len)] Como la primer entrada (axis=0; eje número 0) tiene el score de ser inicio y final lo desapilamos
    return unstacked_logits[0], unstacked_logits[1] 
    # La 1° me devolverá para cada frase del lote y para cada palabra de la frase el score de ser esa palabra de ese lote el incio de la respuesta
    # El 2° me devolverá para cada frase del lote y para cada palabra de la frase el score de ser esa palabra de ese lote el final de la respuesta


## Modelo completo

Construimos una clase que será todo el modelo que juntará la primera parte de BERT con la capa de squad.

In [ ]:
class BERTSquad(tf.keras.Model):
    
    def __init__(self,
                 name="bert_squad"):
        super(BERTSquad, self).__init__(name=name)
        
        self.bert_layer = hub.KerasLayer(
            "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
            trainable=True)
        
        self.squad_layer = BertSquadLayer()
    
    # Definimos una función de ayuda que lo que hará será formatear la entrada de BERT porque recordemos que necesitamos tener los id's, las 
    # máscaras y los tokens de frase en ese orden correctamente para la entrada del algoritmo BERT.
    def apply_bert(self, inputs):
        _ , sequence_output = self.bert_layer([inputs["input_word_ids"],    # Clave con la que se accede a los identificadores de cada palabra
                                               inputs["input_mask"],        # Es con el que se accede a la máscara de aleatorización.   
                                               inputs["input_type_ids"]])   # Nos indica si la frase es la número 1 o la número 2
        # Recordemos que la capa de BERT devuelve el vector se corresponde a toda la frase en 1° posición y el que corresponde a cada una de
        # las palabras tokenizadas en 2° posición  
        return sequence_output

    def call(self, inputs):
        seq_output = self.apply_bert(inputs)

        start_logits, end_logits = self.squad_layer(seq_output)
        
        return start_logits, end_logits

# Fase 4: Entrenamiento

## Creación de la IA

In [ ]:
TRAIN_DATA_SIZE = 88641     # Tamaño/longitud de nuestro conjunto de entrenamiento.
# Este es un dataset bastante pesado para entrenar y para poder entrenarlo en google colab sin que nos eche es hacer un pequeño truco y es no 
# utilizar todo el data set para entrenar sino que queremos ver BERT funciona correctamente por lo que definimos:
NB_BATCHES_TRAIN = 2000     # Esto representa aprox un 10% del total de lotes que tenemos 88641/4*.10
BATCH_SIZE = 4              
NB_EPOCHS = 3               # Inclusive 2 podría ser más que suficiente
# Versión modificada del optimizador Adam por Google creado explictamemnte para esta tarea:
INIT_LR = 5e-5              # Ratio de aprendizaje al inicio. Se define con el propósito de que el ratio de aprendizaje sea pequeño al inicio para que pueda ir experimentando y vaya incrementando durante un ratio determinado de pasos:
WARMUP_STEPS = int(NB_BATCHES_TRAIN * 0.1)  # Ratio determinado de pasos que será el 10% del lote 2000*10% = 200

In [ ]:
train_dataset_light = train_dataset.take(NB_BATCHES_TRAIN)

Creamos y compilamos nuestro modelo BERTSquad

In [ ]:
bert_squad = BERTSquad()

Creamos nuestro optimizador Adam modificado por Google.

In [ ]:
optimizer = optimization.create_optimizer(
    init_lr=INIT_LR,
    num_train_steps=NB_BATCHES_TRAIN,
    num_warmup_steps=WARMUP_STEPS)

Creamos nuestra función de perdidas personalizada. Recordemos que nuestra capa final nos devolverá 2 vectores uno en donde nos devuelve la verosimilitud de que un token sea el token de incio y otro considerando cual es el token final. Al final de cuentas suena como que hay que resolver un problema de clasificación pues hay que decidir si es o no el token que buscamos. Por lo tanto, utilizaremos sparse_categorical_crossentropy.

In [ ]:
# labels: etiquetas reales
# model_outputs: salidas del modelo
def squad_loss_fn(labels, model_outputs):
    start_positions = labels['start_positions']
    end_positions = labels['end_positions']
    start_logits, end_logits = model_outputs

    start_loss = tf.keras.backend.sparse_categorical_crossentropy(
        start_positions, start_logits, from_logits=True) 
# from_logits=True: Significa que la respuesta de nuestro modelo, en este caso el segundo parámetro, no es reralmente una probabilidad y es que en 
# ningún momento hemos aplicado una función softmax o algo así que se encargue de que todos los valores esten entre 0 y 1 y la suma de todos sea
# igual a 1. Ahora solo tenemos números de medidas de verosimilitud, pero que en principio serán números cualesquiera, podrían ser positivos 
# o negativos y al aplicar from_logits=True, la función sparse_categorical_crossentropy se encargará de hacer las modificaciones adecuadas 
# y de comparar la respuesta para devovlernos esa pérdida. 

    end_loss = tf.keras.backend.sparse_categorical_crossentropy(
        end_positions, end_logits, from_logits=True)
    
    # Obtenemos la media de ambas perdidas
    total_loss = (tf.reduce_mean(start_loss) + tf.reduce_mean(end_loss)) / 2    
    # reduce_mean: ya que queremos obtener la pérdida de todo el lote. En un principio obtendríamos 4 valores ya que recordemos que son 4 frases 
    # (dado que BATCH_SIZE = 4)

    return total_loss

# Finalmente, como queremos construir nuestro propio bucle de entrenamiento personalizado, nos va a intersar tener un tracking de la función 
# de perdidas (global duarante toda la fase de training) que acabamos de crear y que no es una de las estándar que utilizamos de las de tf.
train_loss = tf.keras.metrics.Mean(name="train_loss")

In [ ]:
next(iter(train_dataset_light))

({'input_mask': <tf.Tensor: shape=(4, 384), dtype=int32, numpy=
  array([[1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>,
  'input_type_ids': <tf.Tensor: shape=(4, 384), dtype=int32, numpy=
  array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>,
  'input_word_ids': <tf.Tensor: shape=(4, 384), dtype=int32, numpy=
  array([[ 101, 2216, 2040, ...,    0,    0,    0],
         [ 101, 2054, 7017, ...,    0,    0,    0],
         [ 101, 2054, 2001, ...,    0,    0,    0],
         [ 101, 2054, 2565, ...,    0,    0,    0]], dtype=int32)>},
 {'end_positions': <tf.Tensor: shape=(4,), dtype=int32, numpy=array([111,  74,  82, 102], dtype=int32)>,
  'start_positions': <tf.Tensor: shape=(4,), dtype=int32, numpy=array([109,  71,  80, 100], dtype=int32)>})

Compilamos el modelo:

In [ ]:
bert_squad.compile(optimizer,
                   squad_loss_fn)

In [ ]:
# Crearemos un sistema de checkpoint en google colab que en caso de que se reinicie la sesión podemos reanudar
# desde elúltimo checkpoint que se haya guardado o incluso más adelante , seguir entrenando con más textos 
# desde donde lo habíamos dejado en lugar de tener que crear la red neuronal desde el inicio.

# Definimos una ruta dentro de mi Google Drive donde se guardarán los checkpoints
checkpoint_path = "./drive/My Drive/Curso de NLP/BERT/ckpt_bert_squad/"

# Guardamos el modelo que queremos guardar autoamticamente siempre que sea posible
ckpt = tf.train.Checkpoint(bert_squad=bert_squad)

# Definimos el manager que será el encargado de guardar los checkpoints en la ruta establecida
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)
# max_to_keep: Es para definir que queremos que siempre se guarden los últimos 5 checkpoints.
# En este caso si quisieramos guardar todos bastaría con poner max_to_keep = 3 pues son 3 epochs

# Las siguientes líneas de código lo que hacen es preguntarle al Checkpoint Manager si hay o no hay 
# último checkpoint
if ckpt_manager.latest_checkpoint:      # esta linea devuelve un None si no hay checkpoint previo (If None:)
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Último checkpoint restaurado!!")

Último checkpoint restaurado!!


## Entrenamiento personalizado

In [ ]:
for epoch in range(NB_EPOCHS):
    print("Inicio del Epoch {}".format(epoch+1))
    start = time.time()
    
    train_loss.reset_states()   # Devuelve toda la función de pérdidas a 0 y así durante un epoch llevar traking solo de este
    
    for (batch, (inputs, targets)) in enumerate(train_dataset_light):
        with tf.GradientTape() as tape:
            model_outputs = bert_squad(inputs)
            loss = squad_loss_fn(targets, model_outputs)
        
        gradients = tape.gradient(loss, bert_squad.trainable_variables)     # Se obtiene el gradiente solo para las variables entrenables de BERT
        optimizer.apply_gradients(zip(gradients, bert_squad.trainable_variables))   # juntamos la info de los gradientes y a que variables pertenecen
        
        train_loss(loss)
        
        # Si el lote es múltiplo de 50 que se impriman los siguientes valores:
        if batch % 50 == 0:
            print("Epoch {} Lote {} Pérdida {:.4f}".format(
                epoch+1, batch, train_loss.result()))
        
        # Si el lote es múltiplo de 500 se hará un checkpoint
        if batch % 500 == 0:
            ckpt_save_path = ckpt_manager.save()
            print("Guardando checkpoint para el epoch {} en el directorio {}".format(epoch+1,
                                                                ckpt_save_path))
    print("Tiempo total para entrenar 1 epoch: {} segs\n".format(time.time() - start))

Inicio del Epoch 1
Epoch 1 Lote 0 Pérdida 6.0621
Guardando checkpoint para el epoch 1 en el directorio ./drive/My Drive/Curso de NLP/BERT/ckpt_bert_squad/ckpt-1
Epoch 1 Lote 50 Pérdida 5.8209
Epoch 1 Lote 100 Pérdida 5.2858
Epoch 1 Lote 150 Pérdida 4.6081
Epoch 1 Lote 200 Pérdida 4.1292
Epoch 1 Lote 250 Pérdida 3.7406
Epoch 1 Lote 300 Pérdida 3.4795
Epoch 1 Lote 350 Pérdida 3.3128
Epoch 1 Lote 400 Pérdida 3.1128
Epoch 1 Lote 450 Pérdida 2.9574
Epoch 1 Lote 500 Pérdida 2.8129
Guardando checkpoint para el epoch 1 en el directorio ./drive/My Drive/Curso de NLP/BERT/ckpt_bert_squad/ckpt-2
Epoch 1 Lote 550 Pérdida 2.6840
Epoch 1 Lote 600 Pérdida 2.6034
Epoch 1 Lote 650 Pérdida 2.5247
Epoch 1 Lote 700 Pérdida 2.4478
Epoch 1 Lote 750 Pérdida 2.3795
Epoch 1 Lote 800 Pérdida 2.3212
Epoch 1 Lote 850 Pérdida 2.2794
Epoch 1 Lote 900 Pérdida 2.2269
Epoch 1 Lote 950 Pérdida 2.1805
Epoch 1 Lote 1000 Pérdida 2.1184
Guardando checkpoint para el epoch 1 en el directorio ./drive/My Drive/Curso de NLP/BER

# Fase 5: Evaluación

## Preparación de la evaluación

Get the dev set in the session

In [ ]:
eval_examples = read_squad_examples(
    "/content/drive/My Drive/Curso de NLP/BERT/squad_data/dev-v1.1.json",
    is_training=False,
    version_2_with_negative=False)

Define the function that will write the tf_record file for the dev set

In [ ]:
eval_writer = FeatureWriter(
    filename=os.path.join("/content/drive/My Drive/Curso de NLP/BERT/squad_data/",
                          "eval.tf_record"),
    is_training=False)
# FeatureWriter: a la hr de llevar a cabo la parte de la evaluación necesitamos contar con features, los
# cuales no son más que protocolos que tf utiliza a la hr de trabajar. Son herramientas que se utlizan para
# proporcionar la información o almacenar la información. En otras palabras es una convención si 
# queremos usar tf squad.

Create a tokenizer for future information needs

In [ ]:
my_bert_layer = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
    trainable=False)
vocab_file = my_bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = my_bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

Define the function that add the features (feature is a protocol in tensorflow) to our eval_features list

In [ ]:
# Como he dicho los features no son más que pedazos de info que siguen un protocolo, que siguen una
# convención y por tanto solo lo convierto al formato que me intersa.
def _append_feature(feature, is_padding):
    if not is_padding:
        eval_features.append(feature)
    eval_writer.process_feature(feature)

Create the eval features and the writes the tf.record file

In [ ]:
eval_features = []
dataset_size = convert_examples_to_features(
    examples=eval_examples,
    tokenizer=tokenizer,
    max_seq_length=384,     # este valor varia dependiendo los datos
    doc_stride=128,
    max_query_length=64,
    is_training=False,
    output_fn=_append_feature,
    batch_size=4)

In [ ]:
eval_writer.close()

Load the ready-to-be-used dataset to our session

In [ ]:
BATCH_SIZE = 4

eval_dataset = create_squad_dataset(
    "/content/drive/My Drive/Curso de NLP/BERT/squad_data/eval.tf_record",
    384,#input_meta_data['max_seq_length'],
    BATCH_SIZE,
    is_training=False)

## Llevar a cabo las prediccioness

Definir un cierto tipo de colección (como un diccionario). Esto nos será util ya que como seguiremos trabajando con lotes en un momento dado, vamos querer crear una función que devuelva los elementos de salida por lote uno detrás de otro. Entonces, para ello nos va ir muy bien que los elementos vayan nombrados.

In [ ]:
# Creamos una tupla nombrado lo que nos permitirá crear tuplas dodne cada elemento ahora tenga un nombre
# en lugar de las tuplas estandár a las que se accede por posición.
RawResult = collections.namedtuple("RawResult",
                                   ["unique_id", "start_logits", "end_logits"])

Devuelve cada elemento del lote de salida, uno por uno. Dado un elemento del lote de salida lo va a iterar en este formato para devolverme empaquetada la información que me interesa. Enotras palabras de la salida del modelos solo me quedaré con unique_ids, start_logits y end_logits. Lo que hace zip es que empaqueta la info en 3 listas con una misma longitud.

In [ ]:
def get_raw_results(predictions):
    for unique_ids, start_logits, end_logits in zip(predictions['unique_ids'],
                                                    predictions['start_logits'],
                                                    predictions['end_logits']):
        yield RawResult(
            unique_id=unique_ids.numpy(),       # lo convertimos a formato numpy para poder manejarlo más comodamente
            start_logits=start_logits.numpy().tolist(), # lo convertimos a lista ya que más adelante utilizaré una de las funciones de Google que aceptan como entrada a estas como listas
            end_logits=end_logits.numpy().tolist())
# si no conocen yield: significa que esta función realmente actuará como si fuera un iterador, ie, podemos
# hacer un bucle sobre este objeto cada vez que la llamemos. Funcionará exactamente como un iterador, de
# modo que podemos hacer un bucle a tráves del objeto get_raw_results de una predicción e ir obteniendo cada 
# una de la entradas. De ahí esa gracia de yield porque devolverá un elemento y esperará a que un bucle pida
# la siguiente observación. Entonces, como la predicción nos vendrá en lotes yo le puedo dar todo el lote 
# e iterar directamente sobre el resultado de get_raw_results para devolver empaquetado el id, el loggit 
# de entrada y el loggit de salida.

Hacemos nuestras predicciones

In [ ]:
all_results = []
for count, inputs in enumerate(eval_dataset):
    x, _ = inputs   # nos quedamos como entrada solo la pregunta y nos deshacemos de la respuesta
    unique_ids = x.pop("unique_ids")    # Quitamos el identificador ya que si se lo doy a la capa de BERT no sabrá muy bien que hacer con él
    start_logits, end_logits = bert_squad(x, training=False)
    output_dict = dict(
        unique_ids=unique_ids,  # Fijemonos que he hecho pop pero lo he guardado en una variable. Me quedado con una referencia.
        start_logits=start_logits,
        end_logits=end_logits)
    for result in get_raw_results(output_dict): # Como get_raw_results es un iterador no me dará ningun problema
        all_results.append(result)
    if count % 100 == 0:
        print("{}/{}".format(count, 2709))

0/2709
100/2709
200/2709
300/2709
400/2709
500/2709
600/2709
700/2709
800/2709
900/2709
1000/2709
1100/2709
1200/2709
1300/2709
1400/2709
1500/2709
1600/2709
1700/2709
1800/2709
1900/2709
2000/2709
2100/2709
2200/2709
2300/2709
2400/2709
2500/2709
2600/2709
2700/2709


Escribimos nuestras predicciones en un fichero JSON que funcionará con el script de evaluación. El que realmente nos intersa es el primero los otros dos son ficheros que le hace falta Google y hay que mencionar el path en donde están si no pues fallaría la función.

In [ ]:
output_prediction_file = "/content/drive/My Drive/Curso de NLP/BERT/squad_data/predictions.json"
output_nbest_file = "/content/drive/My Drive/Curso de NLP/BERT/squad_data/nbest_predictions.json"
output_null_log_odds_file = "/content/drive/My Drive/Curso de NLP/BERT/squad_data/null_odds.json"

write_predictions(
    eval_examples,  
    eval_features,  # formato interno que habíamos meniconado que sería necesario para poder crear el fichero correctamente
    all_results,    # los resultados que hemos obtenido 
    20,             # No sé que sea pero son necesarios
    30,             # No sé que sea pero son necesarios
    True,           # Ya que hemos echo uso del lower case en la fase de tokenización.
    output_prediction_file, 
    output_nbest_file,
    output_null_log_odds_file,
    verbose=False)  # Para que no se nos impirma mucha basura en la pantalla mientras corre

## Predicción casera

### Creación del diccionario de entrada

Concatenamos la pregunta y el contexto, separados por `["SEP"]`, tras la tokenización, tal cual como lo hicimos con el conjunto de entrenamiento.

Lo importante a recordar es que queremos que nuestra respuesta empiece y termine con una palabra real. Por ejemplo, la palabra "ecologically" es tokenizada como `["ecological", "##ly"]`, y si el token de fin es `["ecological"]` queremos usar la palabra "ecologically" como palabra final (del mismo modo si el token de fin es`["##ly"]`). Por eso, empezamos dividiendo nuestro contexto en palabras, y luego pasamos a tokens, recordando qué token se corresponde con qué palabra (ver la función `tokenize_context()` para más detalle).

#### Útiles varios

In [ ]:
my_bert_layer = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
    trainable=False)
vocab_file = my_bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = my_bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [ ]:
def is_whitespace(c):
    '''
    Indica si un cadena de caracteres se corresponde con un espacio en blanco / separador o no.
    '''
    if c == " " or c == "\t" or c == "\r" or c == "\n" or ord(c) == 0x202F:
        return True
    return False

In [ ]:
# Lo que haremos aqui es enchufarle todo el texto y tendremos en doc_tokens los tokens del documento
# y si previamente procedían de un espacio en blanco (prev_is_whitespace) y para cada caracter del texto 
# iremos leyendo, leyendo si es espacio en blanco (is_whitespace(c)) acabo de encontrar un espacio 
# en blanco prev_is_whitespace = True. Si no es un espacio en blanco (else), entonces depende, si
# previamente teníamos un espacio en blanco (if prev_is_whitespace) añadimos un token a los tokens 
# del documento y si no lo único que hacemos es ir apendizando caracteres al documento para que 
# sea unicamente cuando leo un espacio en blanco que todos los document tokens han sido recopilados
# que hemos localizado todas las palabras y por lo tanto podemos añadirlo al token. En pocas palabras,
# esto lo que nos va a hacer es romper el texto en una lista de palabras pero gestionada por mi no 
# gestionada por el tokenizador.
def whitespace_split(text):
    '''
    Toma el texto y devuelve una lista de "palabras" separadas segun los 
    espacios en blanco / separadores anteriores.
    '''
    doc_tokens = []
    prev_is_whitespace = True
    for c in text:
        if is_whitespace(c):
            prev_is_whitespace = True
        else:
            if prev_is_whitespace:
                doc_tokens.append(c)
            else:
                doc_tokens[-1] += c
            prev_is_whitespace = False
    return doc_tokens

In [ ]:
def tokenize_context(text_words):
    '''
    Toma una lista de palabras (devueltas por whitespace_split()) y tokeniza cada
    palabra una por una. También almacena, para cada nuevo token, la palabra original
    del parámetro text_words.
    '''
    text_tok = []
    tok_to_word_id = []
    for word_id, word in enumerate(text_words):
        word_tok = tokenizer.tokenize(word)
        text_tok += word_tok
        tok_to_word_id += [word_id]*len(word_tok)   # leer *
    return text_tok, tok_to_word_id # Devolvemos los tokens del texto y los identificadores de palabras que corresponden a cada uno de los tokens 
# *: añadimos el identificador de la palabra len(word_tok) veces de modo que si una sola palabra pasa a 
# ser 3 tokens porque tiene un prefijo en el cuerpo y un sufijo pues guardaremos 3 veces su identificador.
# Este es el truco para tener una biyección en todo momento de palabra y token o tokens respectivos. Esta
# es la razón del porque es necessario whitespace_split y tokenize_context

Necesitamos crear las 3 entradas diferentes para cada oración.

In [ ]:
# Devulve los identificadores para cada token
def get_ids(tokens):
    return tokenizer.convert_tokens_to_ids(tokens)

# Devulve todo lo que no es el caracter de padding 
def get_mask(tokens):
    return np.char.not_equal(tokens, "[PAD]").astype(int)

# Devulve 0 o 1 depeniendo de si esta antes o después el separador o incluso si hubiera más separadores
# me iría combinando 0 o 1 para cada una de las palabras que hubiera de entre los separadores
def get_segments(tokens):
    seg_ids = []
    current_seg_id = 0
    for tok in tokens:
        seg_ids.append(current_seg_id)
        if tok == "[SEP]":
            current_seg_id = 1-current_seg_id # Convierte 1 en 0 y viceversa
    return seg_ids

Creamos nuestro diccionario de entradas de modo que le damos una pregunta y un contexto ynos devolverá un diccionario con los 3 elementos que le hacen falta al modelo. A saber, los tokens o los id's de tokens las máscaras y el identificador de frase. También devolverá las palabras del contexto (context_words) además id's de los tokens de contextos (context_tok_to_word_id) y la longitud de los tokens.

In [ ]:
def create_input_dict(question, context):
    '''
    Take a question and a context as strings and return a dictionary with the 3
    elements needed for the model. Also return the context_words, the
    context_tok to context_word ids correspondance and the length of
    question_tok that we will need later.
    '''
    question_tok = tokenizer.tokenize(my_question)

    context_words = whitespace_split(context)
    context_tok, context_tok_to_word_id = tokenize_context(context_words)

    input_tok = question_tok + ["[SEP]"] + context_tok + ["[SEP]"]
    input_tok += ["[PAD]"]*(384-len(input_tok)) # in our case the model has been
                                                # trained to have inputs of length max 384
    # Generamos la entrada para BERT                                            
    input_dict = {}
    input_dict["input_word_ids"] = tf.expand_dims(tf.cast(get_ids(input_tok), tf.int32), 0)
    input_dict["input_mask"] = tf.expand_dims(tf.cast(get_mask(input_tok), tf.int32), 0)
    input_dict["input_type_ids"] = tf.expand_dims(tf.cast(get_segments(input_tok), tf.int32), 0)

    return input_dict, context_words, context_tok_to_word_id, len(question_tok)

#### Creación

In [ ]:
my_context = '''Neoclassical economics views inequalities in the distribution of income as arising from differences in value added by labor, capital and land. Within labor income distribution is due to differences in value added by different classifications of workers. In this perspective, wages and profits are determined by the marginal value added of each economic actor (worker, capitalist/business owner, landlord). Thus, in a market economy, inequality is a reflection of the productivity gap between highly-paid professions and lower-paid professions.'''

Neoclassical economics views inequalities in the distribution of income as arising from differences in value added by labor, capital and land. Within labor income distribution is due to differences in value added by different classifications of workers. In this perspective, wages and profits are determined by the marginal value added of each economic actor (worker, capitalist/business owner, landlord). Thus, in a market economy, inequality is a reflection of the productivity gap between highly-paid professions and lower-paid professions.

In [ ]:
#my_question = '''What philosophy of thought addresses wealth inequality?'''
my_question = '''What are examples of economic actors?'''
#my_question = '''In a market economy, what is inequality a reflection of?'''

In [ ]:
my_input_dict, my_context_words, context_tok_to_word_id, question_tok_len = create_input_dict(my_question, my_context)

### Predicción

In [ ]:
start_logits, end_logits = bert_squad(my_input_dict, training=False)

### Interpretación

We remove the ids corresponding to the question and the `["SEP"]` token:

In [ ]:
start_logits_context = start_logits.numpy()[0, question_tok_len+1:] 
end_logits_context = end_logits.numpy()[0, question_tok_len+1:]
# 0: Corresponde al lote númeor 0 ya que solo estamos pasando un texto y question_tok_len+1: elimino 
# los tokens de la pregunta + 1 que corresponde al separador quedando así unicamente con el contexto

First easy interpretation:

In [ ]:
# Del contexto dónde empieza la respuesta, es simplemente buscar de los logits (start_logits_context) la
# posición a la que correpsonde el argumento máximo, en qué posición se encuentra la palbra o el token
# con mayor score y búscala en context_tok_to_word_id. Y este sería el identificador de la palabra de
# inicio de la respuesta
start_word_id = context_tok_to_word_id[np.argmax(start_logits_context)]
end_word_id = context_tok_to_word_id[np.argmax(end_logits_context)]

"Advanced" - making sure that the start of the answer is before the end:

Para que no ocurre que nos diga que el token de inicio de la respuesta es por ejemplo el token 21 y el final de la respuesta es el 14. Esta es la versión mejorada con un doble bucle y busca el par de respuestas cuyo argumento máximo cumpla que el token de inicio sea anterior al token final.

In [ ]:
pair_scores = np.ones((len(start_logits_context), len(end_logits_context)))*(-1E10)
for i in range(len(start_logits_context-1)):
    for j in range(i, len(end_logits_context)):
        pair_scores[i, j] = start_logits_context[i] + end_logits_context[j]
pair_scores_argmax = np.argmax(pair_scores)

In [ ]:
start_word_id = context_tok_to_word_id[pair_scores_argmax // len(start_logits_context)] # corresponde a la fila
end_word_id = context_tok_to_word_id[pair_scores_argmax % len(end_logits_context)]      # corresponde a la columna

Final answer:

In [ ]:
predicted_answer = ' '.join(my_context_words[start_word_id:end_word_id+1])
print("The answer to:\n" + my_question + "\nis:\n" + predicted_answer)

The answer to:
What are examples of economic actors?
is:
(worker, capitalist/business owner, landlord).


In [ ]:
from IPython.core.display import HTML
display(HTML(f'<h2>{my_question.upper()}</h2>'))
marked_text = str(my_context.replace(predicted_answer, f"<mark>{predicted_answer}</mark>"))
display(HTML(f"""<blockquote> {marked_text} </blockquote>"""))

#### Reto Final

In [ ]:
my_context = '''
Coronavirus disease 2019 is an infectious disease caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2). It was first identified in December 2019 in Wuhan, Hubei, China, and has resulted in an ongoing pandemic.
Common symptoms include fever, cough, fatigue, shortness of breath, and loss of smell and taste.While most people have mild symptoms, some people develop acute respiratory distress syndrome (ARDS) possibly precipitated by cytokine storm, multi-organ failure, septic shock, and blood clots. The time from exposure to onset of symptoms is typically around five days, but may range from two to fourteen days.
The virus is spread primarily via nose and mouth secretions including small droplets produced by coughing,[a] sneezing, and talking. The droplets usually do not travel through air over long distances. However, those standing in close proximity may inhale these droplets and become infected.[b] People may also become infected by touching a contaminated surface and then touching their face. The transmission may also occur through smaller droplets that are able to stay suspended in the air for longer periods of time in enclosed spaces.'''

my_question = '''What are the common symptoms of the disease?'''

my_input_dict, my_context_words, context_tok_to_word_id, question_tok_len = create_input_dict(my_question, my_context)

start_logits, end_logits = bert_squad(my_input_dict, training=False)

pair_scores = np.ones((len(start_logits_context), len(end_logits_context)))*(-1E10)
for i in range(len(start_logits_context-1)):
    for j in range(i, len(end_logits_context)):
        pair_scores[i, j] = start_logits_context[i] + end_logits_context[j]
pair_scores_argmax = np.argmax(pair_scores)

start_word_id = context_tok_to_word_id[pair_scores_argmax // len(start_logits_context)]
end_word_id = context_tok_to_word_id[pair_scores_argmax % len(end_logits_context)]

predicted_answer = ' '.join(my_context_words[start_word_id:end_word_id+1])


from IPython.core.display import HTML
display(HTML(f'<h2>{my_question.upper()}</h2>'))
marked_text = str(my_context.replace(predicted_answer, f"<mark>{predicted_answer}</mark>"))
display(HTML(f"""<blockquote> {marked_text} </blockquote>"""))